# TOPIC MODELLING 

### Project Goal: 
To compare customer sentiment regarding attractions at three Disneyland locations (Anaheim, Paris, and Hong Kong) using sentiment analysis, emotion detection, and n-gram associations. To identify keywords using topic modeling that can help Disney determine visitor pain points and improve park experience.<br>
### Dataset:
Dataset is from [Kaggle](https://www.kaggle.com/arushchillar/disneyland-reviews) containing 42,000 reviews from TripAdvisor about three Disneyland branch locations: California, Paris, Hong Kong. here are 19,406 reviews about Anaheim, 13,630 about Paris, and 9,620 about Hong Kong. Data variables include Review_ID, Rating, Year_Month, Reviewer_Location, Review_Text, and Disneyland_Branch.<br>

In [1]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords


import seaborn as sns
import matplotlib.pyplot as plt
from textblob import TextBlob
import nltk
from nltk import FreqDist
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import state_union
from nltk.tokenize import word_tokenize
import spacy
import string
pd.options.mode.chained_assignment = None  # default='warn'
#nltk.download('averaged_perceptron_tagger')

In [2]:
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

## Load Data

In [3]:
Disneyland = pd.read_csv ('DisneylandReviews.csv', encoding='unicode_escape')
Disneyland.head()

,Review_ID,Rating,Year_Month,Reviewer_Location,Review_Text,Branch
0,670772142,4,2019-4,Australia,If you've ever been to Disneyland anywhere you...,Disneyland_HongKong
1,670682799,4,2019-5,Philippines,Its been a while since d last time we visit HK...,Disneyland_HongKong
2,670623270,4,2019-4,United Arab Emirates,Thanks God it wasn t too hot or too humid wh...,Disneyland_HongKong
3,670607911,4,2019-4,Australia,HK Disneyland is a great compact park. Unfortu...,Disneyland_HongKong
4,670607296,4,2019-4,United Kingdom,"the location is not in the city, took around 1...",Disneyland_HongKong


In [4]:
Disneyland.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42656 entries, 0 to 42655
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Review_ID          42656 non-null  int64 
 1   Rating             42656 non-null  int64 
 2   Year_Month         42656 non-null  object
 3   Reviewer_Location  42656 non-null  object
 4   Review_Text        42656 non-null  object
 5   Branch             42656 non-null  object
dtypes: int64(2), object(4)
memory usage: 2.0+ MB


In [5]:
#To select rows whose column value equals 'missing' in 'Year_Month':
missing = Disneyland.loc[Disneyland['Year_Month'] == 'missing']
#keep the missing rows seperate
Disneyland = Disneyland[Disneyland.Year_Month != 'missing']

In [6]:
#get year from the year_month
Disneyland['Year'] = pd.to_datetime(Disneyland['Year_Month']).dt.strftime('%Y')

In [7]:
# Using apply(str) method
Disneyland['Month']=Disneyland['Year_Month'].apply(str)
Disneyland['Month']= Disneyland['Month'].apply(lambda x: x[5:])

In [8]:
#join back the missing in year_month
Disneyland = Disneyland.append(missing)
#Disneyland.tail()

C:\Users\chand\AppData\Local\Temp\ipykernel_10012\470078404.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Disneyland = Disneyland.append(missing)


#### Rename the fields in Branch column
There are 3 fields in Branch column we will rename them for our convenience as following

In [9]:
Disneyland['Branch'] = Disneyland['Branch'].replace({'Disneyland_HongKong': 'HongKong', 
                                                     'Disneyland_California':'California', 
                                                     'Disneyland_Paris':'Paris'})

## Topic Modeling
Topic modeling is a technique to extract the underlying topics from large volumes of text.

We can use topic modeling to extract keywords used in reviews to help Disney identify specific issues that concern park visitors. We can build a dictionary and train our topic model using an unsupervised learning algorithm, LDA (Latent Dirichlet Allocation) to uncover hidden patterns in the text. This will also yield the probabilities of topics appearing in our model. Before building this model, we need to remove punctuations, stopwords, and lemmatized the text. Then, identify best value for k (number of topics) by graphing it against coherence score. Pick ideal k value where the coherence score is at maximum. I used the gensim package for this task.

In [10]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

data = Disneyland.Review_Text.values.tolist()
# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]
data_words = list(sent_to_words(data))
#data_words[:1]

In [11]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['if', 'youve', 'ever', 'been', 'to', 'disneyland', 'anywhere', 'youll', 'find', 'disneyland_hong_kong', 'very', 'similar', 'in', 'the', 'layout', 'when', 'you', 'walk', 'into', 'main_street', 'it', 'has', 'very', 'familiar', 'feel', 'one', 'of', 'the', 'rides', 'its', 'small', 'world', 'is', 'absolutely', 'fabulous', 'and', 'worth', 'doing', 'the', 'day', 'we', 'visited', 'was', 'fairly', 'hot', 'and', 'relatively', 'busy', 'but', 'the', 'queues', 'moved', 'fairly', 'well']


In [12]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [13]:
stop_words = set(stopwords.words('english')) 

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['ve', 'ever', 'disneyland', 'anywhere', 'find', 'similar', 'layout', 'walk', 'main_street', 'familiar', 'feel', 'ride', 'small', 'world', 'absolutely', 'fabulous', 'worth', 'day', 'visit', 'fairly', 'hot', 'relatively', 'busy', 'queue', 'move', 'fairly', 'well']]


In [14]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

#### Compute c_v coherence for various number of topics
use below code


def compute_coherence_values(dictionary, corpus, texts, start, stop):

    coherence_values = []
    model_list = []
    for num_topics in range(start, stop):
        model = gensim.models.ldamodel.LdaModel(corpus=corpus, 
                                              num_topics=num_topics,
                                              id2word=id2word,
                                              random_state=90,
                                              alpha='auto',
                                              eta='auto',
                                              per_word_topics=True)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts,
                             dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
    return model_list, coherence_values

start=4
stop=11
model_list, coherence_values = compute_coherence_values(dictionary=id2word, 
                                    corpus=corpus,
                                    texts=data_lemmatized,
                                    start=start, stop=stop)
x = range(start, stop)
plt.figure(figsize=(10,7))
plt.style.use('ggplot')
plt.plot(x, coherence_values, color = "blue",marker=".")
plt.xlabel("Num Topics", size=14)
plt.ylabel("Coherence score", size=14)
plt.title('Number of Topics Based on The Coherence Score',size=18)
plt.show()

In [15]:
#plt.savefig("k_topic.jpg", bbox_inches='tight', dpi = 300)


The ideal number of topics is 4. This gave an a coherence score of 0.38 and perplexity score of -6.94. Applying this k value, we obtain the top words in each topic:

In [20]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=4, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [21]:
# Print the Keyword in the 10 topics
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0, '0.042*"get" + 0.030*"hotel" + 0.025*"take" + 0.023*"pass" + 0.019*"fast" + 0.019*"ticket" + 0.016*"stay" + 0.016*"eat" + 0.015*"use" + 0.012*"restaurant"'), (1, '0.049*"park" + 0.046*"ride" + 0.041*"go" + 0.032*"day" + 0.029*"time" + 0.019*"queue" + 0.018*"get" + 0.014*"food" + 0.013*"good" + 0.013*"wait"'), (2, '0.027*"disney" + 0.023*"see" + 0.022*"great" + 0.022*"kid" + 0.019*"visit" + 0.018*"show" + 0.017*"child" + 0.017*"year" + 0.016*"place" + 0.015*"character"'), (3, '0.033*"staff" + 0.032*"people" + 0.019*"french" + 0.018*"tell" + 0.016*"say" + 0.013*"son" + 0.012*"ask" + 0.011*"rude" + 0.011*"push" + 0.009*"guest"')]


In [22]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -6.9332330206516275

Coherence Score:  0.3860537723382688


In [23]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

C:\Users\chand\AppData\Local\Programs\Python\Python310\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.158558 -0.004348       1        1  44.302168
2     -0.170204 -0.259139       2        1  32.143189
0     -0.059064  0.328902       3        1  16.283002
3      0.387825 -0.065415       4        1   7.271640, topic_info=     Term          Freq         Total Category  logprob  loglift
63   park  50565.000000  50565.000000  Default  30.0000  30.0000
17   ride  47112.000000  47112.000000  Default  29.0000  29.0000
48    get  33958.000000  33958.000000  Default  28.0000  28.0000
133    go  42989.000000  42989.000000  Default  27.0000  27.0000
3     day  32863.000000  32863.000000  Default  26.0000  26.0000
..    ...           ...           ...      ...      ...      ...
69   seem   1076.246910   4656.424509   Topic4  -5.0505   1.1564
194  know   1028.522865   4732.678203   Topic4  -5.0958   1.0948
333  look   1044.212049   5446.022814   Topic4  -5.0807   0.9696
504   put    823.981612   2083.385028   Topic4  -5.3175   1.6936
125  area    848.628343   3619.877479   Topic4  -5.2881   1.1706

[229 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
398       1  0.000792  adult
398       2  0.999097  adult
315       2  0.999966    age
2209      4  0.998879  agree
548       3  0.999787  allow
...     ...       ...    ...
24        2  0.990213  world
1695      4  0.998640  write
397       2  0.999976   year
309       1  0.012570  young
309       2  0.987438  young

[295 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 1, 4])

From our topic model, we were able to obtain top keywords from each topic:
* 44.3% of tokens about park, rides, queue and waiting.
* 32.1% of tokens about positive and fun family park experience.
* 16.3% of tokens about fast passes, restaurant, hotels and bookings.
* 7.3% of tokens about customer service, maintainance, staff and behaviour